# Preditor de Vencedores de Partidas da Premier League

**Aluno:** Luiz Fernando Rabelo (11796893)


O presente notebook faz parte da resolução do Projeto Final da disciplina de Introdução à Ciência de Dados (SME0828). Nele, são analisados dados estatísticos de partidas de futebol do campeonato inglês (Premier League), com o objetivo de construir modelos de aprendizado de máquina que tentam prever qual dos times saiu vencedor, ou se ocorreu um empate em dada partida.

## Descrição do Dataset

Podemos começar a análise descrevendo o conjunto de dados. Foram baixadas estatísticas de partidas de 18 temporadas da Premier League (de 2005 a 2022) da plataforma https://www.football-data.co.uk/. As estatísticas baixadas foram armazenados na pasta "data", no formato CSV. Os atributos estatísticos podem ser divididos entre Estatísticas da Partida, Estatísticas de Resultados e Estatísticas de Casas de Apostas.

**Estatísticas da Partida:**

- **Div:** sigla da liga da partida (neste caso sempre "E0")
- **Date:** data de realização da partida
- **HomeTeam:** nome do time mandante
- **AwayTeam:** nome do time visitante
- **Referee:** nome do árbitro
- **HS:** número de chutes ao gol (totais) do time mandante
- **AS:** número de chutes ao gol (totais) do time visitante
- **HST:** número de chutes ao gol (certos) do time mandante
- **AST:** número de chutes ao gol (certos) do time visitante
- **HC:** número de escanteios do time mandante
- **AC:** número de escanteios do time visitante
- **HF:** número de faltas cometidas pelo time mandante
- **AF:** número de faltas cometidas pelo time visitante
- **HY:** número de cartões amarelos do time mandante
- **AY:** número de cartões amarelos do time visitante
- **HR:** número de cartões vermelhos do time mandante
- **AR:** número de cartões vermelhos do time visitante

**Estatísticas de Resultados:**

- **FTHG:** número gols do time mandante ao final da partida 
- **FTAG:** número gols do time visitante ao final da partida 
- **FTR:** sigla do vencedor da partida (H=mandante, A=time visitante, D=empate) 
- **HTR:** vencedor parcial da partida ao intervalo (H=mandante, A=time visitante, D=empate) 

**Estatísticas de Casas de Apostas:** (\*)

- **AvgH**: média das odds para vitória do time mandante 
- **AvgD**: média das odds para empate
- **AvgA**: média das odds para vitória do time visitante

(\*) O dataset possui diversos outros atributos relacionados à odds de eventos em plataformas de apostas. Mas foram utilizados apenas os atributos supracitados.

## Importação de Bibliotecas

Para o trabalho, foram utilizadas as bibliotecas _pandas_ para manipulação de DataFrames, _numpy_ para o processamento de dados, _seaborn_ e _matplotlib_ para construção de gráficos e _sklearn_ para a preparação, análise de dados e construção de modelos de aprendizado.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

## Leitura dos Dados

Começamos definindo o intervalo de temporadas de interesse. Cada temporada se inicia no começo do 2° semestre de um ano e termina no fim do 1° semestre do ano seguinte.

In [ ]:
ANO_INICIAL = 2005
ANO_FINAL = 2023

Em seguida, guardamos os dados de todas as temporadas no dataframe `dados_pl`, armazenando também as informações lidas segmentadas por temporada no dicionário `dados_pl_por_temporada`.

In [ ]:
dados_pl = None
dados_pl_por_temporada = {}

for ano in range(ANO_INICIAL, ANO_FINAL):
    temporada_atual = f'{ano}-{ano+1}'
    dados_pl_por_temporada[temporada_atual] = pd.read_csv(f'./data/english-pl-{temporada_atual}.csv', encoding='unicode_escape')
    if dados_pl is None:
        dados_pl = dados_pl_por_temporada[temporada_atual]
    else:
        dados_pl = pd.concat([dados_pl, dados_pl_por_temporada[temporada_atual]], ignore_index=True)

print('Shape do dataset original lido:', dados_pl.shape)

## Análises Iniciais

### Distribuição dos Resultados Históricos

Vamos, inicialmente, visualizar a distribuição dos possíveis resultados (H = time mandante vencedor; D = empate; A = time visitante vencedor) das partidas ao longo da história.

In [ ]:
contagens_resultados = dados_pl['FTR'].value_counts()

percentuais_resultados = [
    100 * contagens_resultados[r] / sum(contagens_resultados.values) for r in contagens_resultados.keys()
]

plt.figure(figsize=(10,6))
plt.bar(contagens_resultados.keys(), contagens_resultados.values, color='darkcyan', ec='black')

for i in range(len(percentuais_resultados)):
    plt.text(i, contagens_resultados[i] + 1, f'{percentuais_resultados[i]:.2f}%', ha='center', va='bottom')

plt.title('Resultados dos Jogos ao Longo da História', fontdict={'size': 12, 'weight': 'bold'})
plt.xlabel('Resultado da Partida', fontsize=12)
plt.ylabel('Número de ocorrências', fontsize=12)
plt.show()

No futebol, o chamado "Fator Casa" é bastante conhecido. Em resumo, esse fator sugere que o time mandante de uma partida, geralmente possui certas vantagens sobre o time visitante, podendo ser citadas:

- maior conhecimento do gramado da partida (embora as diferenças no tipo da grama e nas dimensões dos gramados de partidas oficiais sejam pequenas);
- recepção da maior parcela da torcida presente no estádio (a qual ajuda a motivar o time mandante e a pressionar o visitante);
- menor deslocamento para o estádio no período que antecede o jogo (dependendo do jogo, o time visitante deve viajar por quilômetros até o local da partida, e esse deslocamento pode gerar um pequeno desgaste físico para os jogadores).

O Fator Casa, de fato, parece influenciar os resultados das partidas, uma vez que, em quase metade das partidas do conjunto de dados, o time mandante saiu vencedor. 

### Fator Casa por Temporada

Se prosseguirmos com a análise do Fator Casa, agora analisando a pontuação dos times por temporada, chegaremos em um fato interessante: o fator casa não foi tão determinante, ao menos não como nos outros anos, nos resultados das partidas ocorridas durante a pandemia causada pela COVID-19.

In [ ]:
pontuacoes_mandantes = []
temporadas = []
cores = []

for ano in range(ANO_INICIAL, ANO_FINAL):
    temporada_atual = f'{ano}-{ano+1}'
    pontos_vitorias_m = dados_pl_por_temporada[temporada_atual]['FTR'].value_counts()['H'] * 3
    pontos_empates = dados_pl_por_temporada[temporada_atual]['FTR'].value_counts()['D'] * 1
    pontuacoes_mandantes.append(pontos_vitorias_m + pontos_empates)
    temporadas.append(temporada_atual)
    cores.append('darkred' if ano == 2020 else 'forestgreen')

media_pontos_mandantes = sum(pontuacoes_mandantes) / len(pontuacoes_mandantes)

plt.figure(figsize=(10, 6))
plt.bar(temporadas, pontuacoes_mandantes, color=cores, ec='black')
plt.xticks(rotation=75)
plt.title('Pontuações dos Times Como Mandantes por Temporada', fontdict={'size': 12, 'weight': 'bold'})
plt.xlabel('Temporada', fontsize=12)
plt.ylabel('Pontuação', fontsize=12)
plt.axhline(y=media_pontos_mandantes, c='darkorange', ls='--', label='Média Histórica')
plt.legend()
plt.show()

In [ ]:
pontuacoes_visitantes = []
temporadas = []
cores = []

for ano in range(ANO_INICIAL, ANO_FINAL):
    temporada_atual = f'{ano}-{ano+1}'
    pontos_vitorias_v = dados_pl_por_temporada[temporada_atual]['FTR'].value_counts()['A'] * 3
    pontos_empates = dados_pl_por_temporada[temporada_atual]['FTR'].value_counts()['D'] * 1
    pontuacoes_visitantes.append(pontos_vitorias_v + pontos_empates)
    temporadas.append(temporada_atual)
    cores.append('darkred' if ano == 2020 else 'forestgreen')

media_pontos_visitantes = sum(pontuacoes_visitantes) / len(pontuacoes_visitantes)

plt.figure(figsize=(10, 6))
plt.bar(temporadas, pontuacoes_visitantes, color=cores, ec='black')
plt.xticks(rotation=75)
plt.title('Pontuações dos Times Como Visitantes por Temporada', fontdict={'size': 12, 'weight': 'bold'})
plt.xlabel('Temporada', fontsize=12)
plt.ylabel('Pontuação', fontsize=12)
plt.axhline(y=media_pontos_visitantes, c='darkorange', ls='--', label='Média Histórica')
plt.legend()
plt.show()

Essa "anormalidade" pode ser explicada pelas questões sanitárias da temporada 2020-2021, que iniciou com portões fechados para torcida e assim permaneceu pela maior parte do tempo, com exceção de um breve período em que os clubes puderam convidar um número limitado de torcedores para voltar aos estádios. O desvio nas pontuações dos times em casa fortificou a opinião de muitos especialistas em futebol de que a presença da torcida nas partidas exerce forte influência positiva para o time mandante.

Os detalhes de como a COVID-19 afetou o campeonato inglês podem ser conferidos em https://www.premierleague.com/news/1682374.

### Desempenho dos Maiores Clubes

Outra análise interessante de ser feita é a do desempenho geral dos grandes clubes do Campeonato Inglês. Na Premier League, existem 6 clubes considerados de elite, são eles: Arsenal, Chelsea, Liverpool, Manchester United, Manchester City e Tottenham. Esse grupo de times recebeu o nome de "Big Six" na Inglaterra. Podemos comparar o desempenho dos clubes do Big Six com os demais.

Primeiro, definimos a lista dos nomes dos clubes pertencentes ao grupo de elite.

In [ ]:
times_big_six = ['Arsenal', 'Chelsea', 'Liverpool', 'Man United', 'Man City', 'Tottenham']

Então definimos filtros para os casos de pontuação por vitória e por empate para cada grupo.

In [ ]:
condicao_vitoria_b6 = (
    (dados_pl['HomeTeam'].isin(times_big_six)) & (dados_pl['FTR'] == 'H') | 
    (dados_pl['AwayTeam'].isin(times_big_six)) & (dados_pl['FTR'] == 'A')
)

condicao1_empate_b6 = (
    (dados_pl['HomeTeam'].isin(times_big_six)) ^ (dados_pl['AwayTeam'].isin(times_big_six)) &
    (dados_pl['FTR'] == 'D')
)

condicao2_empate_b6 = (
    (dados_pl['HomeTeam'].isin(times_big_six)) & (dados_pl['AwayTeam'].isin(times_big_six)) &
    (dados_pl['FTR'] == 'D')
)

condicao_vitoria_demais = (
    (~(dados_pl['HomeTeam'].isin(times_big_six))) & (dados_pl['FTR'] == 'H') | 
    (~(dados_pl['AwayTeam'].isin(times_big_six))) & (dados_pl['FTR'] == 'A')
)

condicao1_empate_demais = (
    (~(dados_pl['HomeTeam'].isin(times_big_six))) ^ (~(dados_pl['AwayTeam'].isin(times_big_six))) &
    (dados_pl['FTR'] == 'D')
)

condicao2_empate_demais = (
    (~(dados_pl['HomeTeam'].isin(times_big_six))) & (~(dados_pl['AwayTeam'].isin(times_big_six))) &
    (dados_pl['FTR'] == 'D')
)

Assim, aplicamos os filtros no dataset, contabilizando as quantidades de vitória e de empates para cada grupo. Vale destacar que, se 2 times do mesmo grupo se enfrentarem e houver empate, o empate deve ser contabilizado 2 vezes, pois ambos os times pontuarão nesse cenário.

In [ ]:
vitorias_b6 = dados_pl[condicao_vitoria_b6].shape[0]
empates_b6 = dados_pl[condicao1_empate_b6].shape[0] + 2 * dados_pl[condicao2_empate_b6].shape[0]

vitorias_demais = dados_pl[condicao_vitoria_demais].shape[0]
empates_demais = dados_pl[condicao1_empate_demais].shape[0] + 2 * dados_pl[condicao2_empate_demais].shape[0]

Mostramos, então, a divisão das pontuações por grupo (3 pontos por vitória e 1 ponto por empate) acumuladas ao longo da hisória.

In [ ]:
pontuacoes = {
    'Times Big 6': vitorias_b6 * 3 + empates_b6 * 1,
    'Demais Times':  vitorias_demais * 3 + empates_demais * 1,
}

plt.figure(figsize=(8,8))
plt.pie(pontuacoes.values(), labels=pontuacoes.keys(), startangle=90, autopct='%1.1f%%', explode=[0, 0.05], colors=['darkcyan', 'coral'])
plt.title('Pontuações dos Times por Grupo (Big 6 vs Demais)', fontdict={'size': 12, 'weight': 'bold'});

Dos 42 times que disputaram a Premiere League durante o período, 6 foram responsáveis pelo significativo percentual de quase 40% da pontuação histórica.

## Preparação dos Dados

### Seleção dos Atributos de Interesse

Vamos iniciar a preparação dos dados para a contrução dos modelos. Começamos selecionando os atributos de interesse. Nessa etapa, é descartada a divisão da liga (que é sempre a mesma dentro do conjunto de dados), o nome do árbitro da partida (que, em tese, não deve influenciar no resultado) e os atributos de casas de apostas, com exceção das odds para cada evento relacionado ao resultado, calculadas no início da partida.

In [ ]:
colunas_interesse = [
    'Date', 'HomeTeam', 'AwayTeam', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'AvgH', 'AvgD', 'AvgA', 'HTR', 'FTR'
]

colunas_nao_interesse = [nome_coluna for nome_coluna in dados_pl.columns if nome_coluna not in colunas_interesse]

dados_pl.drop(columns=colunas_nao_interesse, inplace=True)
print('Shape do dataset filtrado:', dados_pl.shape)

### Verificação de Valores Nulos

Veremos quais são as colunas mais afetadas por valores nulos.

In [ ]:
dados_pl.isnull().sum().sort_values(ascending=False)

Como os dados de odds de apostas esportivas foram disponibilizados apenas a partir de 2019, a maior parte dessa coluna possui valores vazios. Isso é um problema, pois as chances de vitória ao início das partidas para cada time são difíceis de ser preenchidas por uma simples regressão, e, nesse sentido, para fins de simplificação, vamos considerar que as probabilidades para cada evento sejam de:

- vitória do mandante: $p_H = \frac{1}{2}$
- empate: $p_D = \frac{1}{4}$
- vitória do visitante: $p_A = \frac{1}{4}$

Assim, podemos determinar as odds exclusivamente em função da probabilidade de cada evento, sem considerar o risco de mercado, como $odd_H\frac{1}{\frac{1}{2}} = 2$, $odd_D = odd_A = \frac{1}{\frac{1}{4}} = 4$.

Preenchemos, então, as odds faltantes no conjunto de dados.

In [ ]:
dados_pl['AvgH'].fillna(2, inplace=True)
dados_pl['AvgD'].fillna(4, inplace=True)
dados_pl['AvgA'].fillna(4, inplace=True)

Os outros atributos que ainda possuem valores nulos estão associados à um único registro do dataset. Sendo assim, podemos optar por apenas removê-lo.

In [ ]:
dados_pl.dropna(inplace=True)
dados_pl.shape

Ao todo, 5321 registros tiveram as colunas de odds alteradas e 1 registro foi excluído.

### Conversão da Data

A data de realização da partida, em si, pode não parecer tão relevante para influenciar em seu resultado. Entretanto, como mostrado na análise inicial, o Fator Casa teve um impacto diferente durante a pandemia. Podemos então, em vez de utilizar a data específica da partida, categoriza-la como pandêmica ou normal.

In [ ]:
is_data_pandemica = lambda data: \
    int(data.split('/')[2]) == 2020 and int(data.split('/')[1]) >= 9 or \
    int(data.split('/')[2]) == 2021 and int(data.split('/')[1]) <= 5

dados_pl['Date'] = dados_pl['Date'].apply(
    lambda data: 'pandemic' if is_data_pandemica(data) else 'regular'
)

Pelo período de análise, a maioria das datas foram regulares.

In [ ]:
value_counts_date = dados_pl['Date'].value_counts()

qtd_datas_pandemicas_total = value_counts_date['pandemic']
qtd_datas_nao_pandemicas_total = value_counts_date['regular']

print(qtd_datas_pandemicas_total, 'datas pandêmicas')
print(qtd_datas_nao_pandemicas_total, 'datas não pandêmicas')

### Separação dos Times em Grupos

Como mostrado, também na análise inicial, os times principais do campeonato são responsáveis por um percentual significativo das pontuações históricas. Assim, é possível dividir os times entre pertencentes ao Big Six ou aos outros.

In [ ]:
dados_pl['HomeGroup'] = dados_pl['HomeTeam'].apply(
    lambda team: 'Big6' if team in times_big_six else 'Others'
)

dados_pl['AwayGroup'] = dados_pl['AwayTeam'].apply(
    lambda team: 'Big6' if team in times_big_six else 'Others'
)

dados_pl.drop(columns=['HomeTeam', 'AwayTeam'], inplace=True)

### Conversão do Resultado da Partida

Os possíveis resultados das partidas (H = time mandante vencedor; D = empate; A = time visitante vencedor) podem ser convertidos de valores textuais para inteiros, tanto para o resultado parcial (ao intervalo) quanto para o resultado final.

In [ ]:
DERROTA_MANDANTE, EMPATE, VITORIA_MANDATE = 0, 1, 2

dados_pl['FTR'].replace({'A': DERROTA_MANDANTE, 'D': EMPATE, 'H': VITORIA_MANDATE}, inplace=True)
dados_pl['HTR'].replace({'A': DERROTA_MANDANTE, 'D': EMPATE, 'H': VITORIA_MANDATE}, inplace=True)

### Separação dos Conjuntos de Treinamento e de Teste

A maior parte dos jogos terminou com o time mandante vencedor. Vamos armazenar as proporções de vitória e derrota dos mandantes.

In [ ]:
resultados_totais = dados_pl['FTR'].value_counts()

qtd_vitorias_mandantes_total = resultados_totais[VITORIA_MANDATE]
qtd_derrotas_mandantes_total = resultados_totais[DERROTA_MANDANTE]

proporcao_vitorias_mandantes_total = qtd_vitorias_mandantes_total / sum(resultados_totais.values)
proporcao_derrotas_mandantes_total = qtd_derrotas_mandantes_total / sum(resultados_totais.values)

print('Proporção de vitórias de mandantes:', proporcao_vitorias_mandantes_total)
print('Proporção de derrotas de mandantes:', proporcao_derrotas_mandantes_total)

Como as classes são desbalanceadas, vamos seguir as proporções armazenadas para dividir os conjuntos de treinamento e de teste, respeitando a distribuição das classes com uma tolerância de 1%.

In [ ]:
PERCENTUAL_TREINAMENTO = 0.8
SEMENTE_ALEATORIA = 42

np.random.seed(SEMENTE_ALEATORIA)

while True:
    dados_pl['random'] = np.random.rand(dados_pl.shape[0])

    dados_treino = dados_pl[dados_pl['random'] <= PERCENTUAL_TREINAMENTO]
    dados_teste = dados_pl[dados_pl['random'] > PERCENTUAL_TREINAMENTO]

    resultados_treino = dados_treino['FTR'].value_counts()
    
    qtd_vitorias_mandantes_treino = resultados_treino[VITORIA_MANDATE]
    proporcao_vitorias_mandantes_treino = qtd_vitorias_mandantes_treino / sum(resultados_treino.values)

    if not (proporcao_vitorias_mandantes_total - 0.01 <= proporcao_vitorias_mandantes_treino <= proporcao_vitorias_mandantes_total + 0.01):
        continue

    qtd_derrotas_mandantes_treino = resultados_treino[DERROTA_MANDANTE]
    proporcao_derrotas_mandantes_treino = qtd_derrotas_mandantes_treino / sum(resultados_treino.values)

    if not (proporcao_derrotas_mandantes_total - 0.01 <= proporcao_derrotas_mandantes_treino <= proporcao_derrotas_mandantes_total + 0.01):
        continue

    resultados_teste = dados_teste['FTR'].value_counts()
    
    qtd_vitorias_mandantes_teste = resultados_teste[VITORIA_MANDATE]
    proporcao_vitorias_mandantes_teste = qtd_vitorias_mandantes_teste / sum(resultados_teste.values)

    if not (proporcao_vitorias_mandantes_total - 0.01 <= proporcao_vitorias_mandantes_teste <= proporcao_vitorias_mandantes_total + 0.01):
        continue

    qtd_derrotas_mandantes_teste = resultados_teste[DERROTA_MANDANTE]
    proporcao_derrotas_mandantes_teste = qtd_derrotas_mandantes_teste / sum(resultados_teste.values)

    if not (proporcao_derrotas_mandantes_total - 0.01 <= proporcao_derrotas_mandantes_teste <= proporcao_derrotas_mandantes_total + 0.01):
        continue

    proporcao_empates_treino = 1 - (proporcao_vitorias_mandantes_treino + proporcao_derrotas_mandantes_treino)
    proporcao_empates_teste = 1 - (proporcao_vitorias_mandantes_teste + proporcao_derrotas_mandantes_teste)

    dados_treinamento = dados_treino.drop('random', axis=1)
    dados_teste = dados_teste.drop('random', axis=1)

    break

print('Shape de treinamento:', dados_treinamento.shape)
print('Proporção de vitórias de mandantes: ', proporcao_vitorias_mandantes_treino)
print('Proporção de empates:               ', proporcao_empates_treino)
print('Proporção de derrotas de mandantes: ', proporcao_derrotas_mandantes_treino)
print('\nShape de teste:', dados_teste.shape)
print('Proporção de vitórias de mandantes: ', proporcao_vitorias_mandantes_teste)
print('Proporção de empates:               ', proporcao_empates_teste)
print('Proporção de derrotas de mandantes: ', proporcao_derrotas_mandantes_teste)

### Conversão de Variáveis Categóricas

As variáveis categóricas dos conjuntos podem ser convertidas para binárias via One Hot Encoding.

In [ ]:
dados_treinamento = pd.get_dummies(dados_treinamento)
dados_teste = pd.get_dummies(dados_teste)

### Conversão dos Dados para o Formato do Numpy

Como o formato do numpy facilita a manipulação das colunas para a classificação, é apropriado realizar uma conversão para tal formato.

In [ ]:
dados_treinamento_np = dados_treinamento.to_numpy()

Y_treinamento = dados_treinamento_np[:,0]
X_treinamento = dados_treinamento_np[:,1:]

In [ ]:
dados_teste_np = dados_treinamento.to_numpy()

Y_teste = dados_teste_np[:,0]
X_teste = dados_teste_np[:,1:]

### Padronização dos Dados

Alguns algoritmos requerem a padronização dos dados para seu funcionamento. A padronização deve ser aplicada, separadamente (para evitar Data Leakage), aos conjuntos de treinamento e teste.

In [ ]:
X_treinamento = StandardScaler().fit_transform(X_treinamento)
X_teste = StandardScaler().fit_transform(X_teste)

## Análise Exploratória

Podemos fazer uma análise exploratória com o intuito de ir examinar melhor dataset preparado, tentando identificar padrões / tendências / relacionamentos entre os atributos.

### Correlação entre os Atributos

Vamos visualizar a correlação entre os atributos através da Matriz de Correlação.

In [ ]:
corr_treinamento = dados_treinamento.corr()

plt.figure(figsize=(8,8))
plt.imshow(corr_treinamento, cmap='Blues')
plt.colorbar()
plt.xticks(range(len(corr_treinamento)), corr_treinamento.columns, rotation='vertical');
plt.yticks(range(len(corr_treinamento)), corr_treinamento.columns);
plt.title('Correlação entre as Variáveis', fontdict={'size': 12, 'weight': 'bold'})
plt.show()

#### Correlações Negativas

Os atributos que soferam transformações via One Hot Encoding são os que estão mais não correlacionados. _Date\_pandemic_ e _Date\_regular_, por serem complementares, possuem correlação 0. O mesmo acontece com as correlações entre os atributos de _Home\_Group_ e _Away\_Group_.

Ademais, os atributos de odds de vitória do time mandante e do time visitante têm uma correlação negativa visível, em virtude de serem determinadas, em maior parte, pelas probabilidades de vitória (que complementares para os eventos).

Assim, dependendo dos resultados obtidos na aplicação dos algoritmos, pode ser interessante remover os atributos completamente (ou altamente) não correlacionados para fazer uma comparação de desempenho.

#### Correlações Positivas

Os atributos de chutes totais (HS e AS) e chutes certos ao gol (HST, AST) apresentam correlação fortemente positiva.

Já com relação ao resultado final, os atributos mais correlacionados são:

- resultado ao intervalo da partida
- quantidade de chutes certos do time mandante
- time mandante/visitante ser pertencente ao Big Six

Vamos analisá-los, separadamente.

**Resultado ao Intervalo da Partida**

In [ ]:
catplot = sns.catplot(x='HTR', hue='FTR', kind='count', data=dados_treinamento, height=6, aspect=1.5, legend=False);
catplot.set(xticklabels=['Time Mandante Perdedor', 'Empate', 'Time Mandante Vencedor'])

plt.title('Comparativo de Resultados ao Meio e ao Fim das Partidas', fontdict={'size': 12, 'weight': 'bold'});
plt.xlabel('Resultado ao Meio da Partida');
plt.ylabel('Quantidade de Ocorrências');
plt.legend(title='Resultado ao Fim da Partida', labels=['Time Mandante Perdedor', 'Empate', 'Time Mandante Vencedor'], loc=(1.01, 0));

Podemos perceber que, se um dos times estiver em vantagem ao intervalo da partida, essa vantagem tende a se manter no resultado final. Já se o resultado parcial for de empate, a distribuição dos resultados finais é mais equilibrada, mas ainda com pequena propensão para se repetir o empate.

**Quantidade de Chutes Certos do Time Mandante**

In [ ]:
dados_treinamento['HST_Range'] = pd.qcut(dados_treinamento['HST'], 5)

catplot = sns.catplot(x='HST_Range', hue='FTR', kind='count', data=dados_treinamento, height=6, aspect=1.5, legend=False);

plt.title('Comparativo de Chutes Certos do Mandante e Resultados das Partidas', fontdict={'size': 12, 'weight': 'bold'});
plt.xlabel('Intervalo de Chutes Certos do Mandante ao Gol');
plt.ylabel('Quantidade de Ocorrências');
plt.legend(title='Resultado ao Fim da Partida', labels=['Time Mandante Perdedor', 'Empate', 'Time Mandante Vencedor'], loc=(1.01, 0));

dados_treinamento.drop('HST_Range', axis=1, inplace=True)

De maneira geral, se o time mandante tiver chutado ao menos 3 vezes diretamente para o gol, ele tendeu a sair vitorioso na partida.

**Pertencimento aos Times do Big 6**

In [ ]:
is_big6_home = dados_treinamento['HomeGroup_Big6'] == 1
is_big6_away = dados_treinamento['AwayGroup_Big6'] == 1
is_big6_both = is_big6_home & is_big6_away

dados_treinamento['Group'] = 'M. Menor x V. Menor'
dados_treinamento.loc[is_big6_home, 'Group'] = 'M. Big6 x V. Menor'
dados_treinamento.loc[is_big6_away, 'Group'] = 'M. Menor x V. Big6'
dados_treinamento.loc[is_big6_both, 'Group'] = 'M. Big6 x V. Big6'

catplot = sns.catplot(x='Group', hue='FTR', kind='count', data=dados_treinamento, height=6, aspect=1.5, legend=False);

plt.title('Comparativo de Grupos de Mandantes x Vistantes por Grupo e Resultados das Partidas', fontdict={'size': 12, 'weight': 'bold'});
plt.xlabel('Grupos do Confronto');
plt.ylabel('Quantidade de Ocorrências');
plt.legend(title='Resultado ao Fim da Partida', labels=['Time Mandante Perdedor', 'Empate', 'Time Mandante Vencedor'], loc=(1.01, 0));

dados_treinamento.drop('Group', axis=1, inplace=True)

Percebemos que os times do Big Six, quando enfrentam times "menores", geralmente saem com a vitória, seja jogando em casa (como mandante) ou fora (como visitante). Nos cenários de duelos entre times do mesmo grupo, o time da casa geralmente saiu vencedor na maioria dos casos.

### Análise de Componentes Principais

Podemos fazer uma análise PCA para apurarmos a variabilidade dos dados. Comecemos com 2 dimensões.

In [ ]:
pca = PCA(n_components=2)
componentes_principais = pca.fit_transform(X_treinamento)

print('Variância por componente principal:', pca.explained_variance_ratio_)

A variância somada das 2 componentes é de $\approx$ 30%. 

Vamos visualizar a variação graficamente em 2 dimensões.

In [ ]:
possiveis_resultados = {
    DERROTA_MANDANTE: 'Derrota Mandante',
    EMPATE: 'Empate',
    VITORIA_MANDATE: 'Vitória Mandante'
}

dados_cps = pd.DataFrame(componentes_principais, columns=['CP1', 'CP2'])
dados_cps['FTR'] = Y_treinamento

plt.figure(figsize=(10, 6))

for resultado in possiveis_resultados.keys():
    indices_to_keep = dados_cps['FTR'] == resultado
    plt.scatter(dados_cps.loc[indices_to_keep, 'CP1'], dados_cps.loc[indices_to_keep, 'CP2'], ec='black', s=50)
    
plt.title('Análise PCA para os Possíveis Resultados - 2 Dimensões')
plt.xlabel('Primeira Componente Principal')
plt.ylabel('Segunda Componente Principal')
plt.legend(possiveis_resultados.values(), loc=(1.01, 0));

Os casos parecem bastante homogêneos. Vamos verificar o que acontece se adicionarmos mais uma dimensão ao PCA.

In [ ]:
pca = PCA(n_components=3)
componentes_principais = pca.fit_transform(X_treinamento)

print('Variância por componente principal:', pca.explained_variance_ratio_)

A variância explicada aumenta um pouco, chegando próxima a 40%.

O novo gráfico ainda deve mostrar certa semelhança de distribuição das classes.

In [ ]:
dados_cps = pd.DataFrame(componentes_principais, columns=['CP1', 'CP2', 'CP3'])
dados_cps['FTR'] = Y_treinamento

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection='3d')

for target in possiveis_resultados.keys():
    indices_to_keep = dados_cps['FTR'] == target
    ax.scatter(dados_cps.loc[indices_to_keep, 'CP1'], dados_cps.loc[indices_to_keep, 'CP2'], dados_cps.loc[indices_to_keep, 'CP3'], ec='black')

ax.set_xlabel('Primeira Componente Principal')
ax.set_ylabel('Segunda Componente Principal')
ax.set_zlabel('Terceira Componente Principal')
ax.set_title('Análise PCA para os Possíveis Resultados - 3 Dimensões')
plt.legend(possiveis_resultados.values());

Como a variância explicada acumulada não foi tão alta para até 3 componentes, podemos expandir a análise para mais componentes principais.

In [ ]:
pca = PCA()
componentes_principais = pca.fit(X_treinamento)

variancia_explicada = pca.explained_variance_
razao_variancia_explicada = pca.explained_variance_ratio_
razao_variancia_acumulada = np.cumsum(razao_variancia_explicada)

numero_componentes = len(variancia_explicada)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.bar(range(1,numero_componentes + 1), razao_variancia_explicada, alpha=0.8, ec='black', align='center')
plt.title('Variância Explicada por Componente', fontdict={'size': 12, 'weight': 'bold'})
plt.xlabel('Número de Componentes', fontsize=12)
plt.ylabel('Proporção de Variância Explicada', fontsize=12)

plt.subplot(1, 2, 2)
plt.plot(range(1,numero_componentes + 1), razao_variancia_acumulada, marker='o', linestyle='--', color='r')
plt.title('Variância Explicada Acumulada', fontdict={'size': 12, 'weight': 'bold'})
plt.xlabel('Número de Componentes', fontsize=12)
plt.ylabel('Proporção Acumulada de Variância Explicada', fontsize=12)

plt.tight_layout()
plt.show()

O relativamente alto número de componentes necessário para a maior proporção de variância explicada acumulada pode indicar que a classificação em questão não será tão simples, mas ainda é possível buscar um bom resultado.

## Seleção e Aplicação de Modelos

Vamos começar definindo 2 funções genéricas para o ajuste e teste dos modelos. Para todos os casos, será utilizado Grid Search com Cross Validation.

A primeira função recebe um modelo como parâmetro, realiza os ajustes e imprime os parâmetros utilizados, bem como a acurácia obtida.

In [ ]:
def ajustar_modelo(modelo):
    modelo.fit(X_treinamento, Y_treinamento)
    print(modelo.best_params_)
    print(f'Acurácia treinamento: {modelo.score(X_treinamento, Y_treinamento):.2f}')

A segunda função recebe um modelo como parâmetro, obtém os resultados preditos pelo modelo (de acordo como o conjunto de teste), calcula as métricas de avaliação e mostra a matriz de confusão das classificações preditas.

In [ ]:
def testar_modelo(modelo):
    Y_previsto = np.array(modelo.predict(X_teste), dtype=int)

    acuracia = accuracy_score(Y_teste, Y_previsto)
    precisao = precision_score(Y_teste, Y_previsto, average='weighted')
    recall = recall_score(Y_teste, Y_previsto, average='weighted')
    f1 = f1_score(Y_teste, Y_previsto, average='weighted')

    print(f'Acurácia teste: {acuracia:.2f}')
    print(f'Precisão teste: {precisao:.2f}')
    print(f'Recall teste: {recall:.2f}')
    print(f'F1 teste: {f1:.2f}')

    cm = confusion_matrix(Y_teste, Y_previsto)

    plt.figure(figsize=(8, 6))

    sns.heatmap(cm, annot=True, fmt='d', cmap='gray', cbar=True)
    plt.title('Matriz de Confusão', fontdict={'size': 12, 'weight': 'bold'})
    plt.xlabel('Valores Preditos', fontsize=12)
    plt.ylabel('Valores Verdadeiros', fontsize=12)
    plt.xticks([DERROTA_MANDANTE + 0.5, EMPATE + 0.5, VITORIA_MANDATE + 0.5], ['Derrota Mandante', 'Empate', 'Vitória Mandante'])
    plt.yticks([DERROTA_MANDANTE + 0.5, EMPATE + 0.5, VITORIA_MANDATE + 0.5], ['Derrota Mandante', 'Empate', 'Vitória Mandante'])
    plt.show()

### K-Nearest Neighbors

Para o KNN, vamos considerar como parâmetros o número de vizinhos de 1 à 49 e as distâncias euclidiana, Manhattan, Minkowski e Chebyshev.

In [ ]:
parametros_grid = {
    'n_neighbors': [k for k in range(50) if k % 2 != 0],
    'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']   
}

modelo_knn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=parametros_grid,
    cv=10,
    scoring='accuracy',
    return_train_score=False,
    verbose=True,
    error_score='raise',
    n_jobs=-1,
)

In [ ]:
ajustar_modelo(modelo_knn)

In [ ]:
testar_modelo(modelo_knn)

### Naive Bayes

Para o Naive Bayes, vamos utilizar o estimador gaussiano, com suavização testada de $1\cdot10^{-9}$ à $1$.

In [ ]:
parametros_grid = {
    'var_smoothing': np.logspace(0, -9, num=100),
}

modelo_bayes = GridSearchCV(
    estimator=GaussianNB(),
    param_grid=parametros_grid,
    cv=10,
    scoring='accuracy',
    return_train_score=False,
    verbose=True,
    error_score='raise',
    n_jobs=-1
)

In [ ]:
ajustar_modelo(modelo_bayes)

In [ ]:
testar_modelo(modelo_bayes)

### Multilayer Perceptron

Para o Multilayer Perceptron, foram testadas 1 camada com 100 neurônios, ativações relu e logística com $\alpha = 0.01$.

In [ ]:
parametros_grid = {
    'hidden_layer_sizes': [(100,)],
    'activation': ['relu', 'logistic'],
    'alpha': [0.01],
}

modelo_perceptron = GridSearchCV(
    estimator=MLPClassifier(solver='sgd', random_state=SEMENTE_ALEATORIA, max_iter=1000),
    param_grid=parametros_grid,
    cv=10,
    scoring='accuracy',
    return_train_score=False,
    verbose=True,
    error_score='raise',
    n_jobs=-1
)

In [ ]:
ajustar_modelo(modelo_perceptron)

In [ ]:
testar_modelo(modelo_perceptron)

### Random Forest

Para o Random Forest, foram considerados estimadores de 100 à 300 (espaçados de 100 em 100), profundidade máxima de 5 à 25 (espaçadas de 5 em 5) e critérios gini e entropia para decisão.

In [ ]:
parametros_grid = {
    'n_estimators': [i * 100 for i in range(1, 4)],
    'max_depth': [i * 5 for i in range(1, 6)],
    'criterion': ['gini', 'entropy'],
}

modelo_forest = GridSearchCV(
    estimator=RandomForestClassifier(random_state=SEMENTE_ALEATORIA),
    param_grid=parametros_grid,
    cv=10,
    scoring='accuracy',
    return_train_score=False,
    verbose=True,
    error_score='raise',
    n_jobs=-1,
)

In [ ]:
ajustar_modelo(modelo_forest)

In [ ]:
testar_modelo(modelo_forest)

### Avaliação dos Desempenhos dos Modelos

No geral, os modelos conseguiram predizer bem os cenários de partidas em que um time saiu vencedor, seja o time mandante ou o time visitante, mas tiveram dificuldade de acertar as partidas que terminaram empatadas.

O resumo das métricas de avaliação obtidas nas predições para cada modelo pode ser verificado na seguinte tabela:

|         Modelo        |                         Parâmetros                        | Acurácia (%) | Precisão (%) | Recall (%) | F1 (%) |
|:---------------------:|:---------------------------------------------------------:|:------------:|:------------:|:----------:|:------:|
|          KNN          |              Distância Manhattan, 47 Vizinhos             |      66      |      63      |     66     |   63   |
|      Naive Bayes      |                   Distribuição Gaussiana                  |      60      |      58      |     60     |   59   |
| Multilayer Perceptron |    Ativação Logística ($\alpha = 0.01)$, 1 Camada com 100 neurônios    |      66      |      63      |     66     |   63   |
|     Random Forest     | Critério Gini, Profundidade Máxima de 10 Nós, 300 Estimadores |      80      |      81      |     80     |   79   |


O modelo que obteve as melhores métricas de avaliação e o único que conseguiu acertar a maior parte dos empates foi o Random Forest. Podemos verificar as importâncias de cada atributo através do modelo.

In [ ]:
nomes_atributos = dados_treinamento.columns[1:]
importacias_atributos = modelo_forest.best_estimator_.feature_importances_

plt.figure(figsize=(10,6))
plt.bar(nomes_atributos, importacias_atributos, color='darkcyan', ec='black')
plt.title('Importâncias dos Atributos Segundo o Random Forest', fontdict={'size': 12, 'weight': 'bold'})
plt.xlabel('Nomes dos Atributos', fontsize=12)
plt.ylabel('Importâncias', fontsize=12)
plt.xticks(rotation=90)
plt.show()

O resultado ao meio da partida (HTR) foi o atributo mais predominante para o modelo, seguido dos atributos de chutes certos ao gol (HST e AST). A pouca influência dos atributos de casas de apostas (AvgH, AvgD, e AvgA) pode ser explicada pela substituição manual da maioria das odds na preparação dos dados, que, por ter sido feita de maneira bastante genérica, pode ter acarretado em uma perda de influência do atributo.

## Re-Preparação dos Dados

Vamos reaplicar os algoritmos utilizando agora os atributos de maior importância e desconsiderando as colunas convertidas com One Hot Encoding e a Odd de vitória do time visitante (complementar às outras odds).

### Seleção dos Atributos de Interesse

In [ ]:
colunas_interesse = [
    'HomeGroup_Big6', 'AwayGroup_Big6', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'AvgH', 'AvgD', 'HTR', 'FTR'
]

colunas_nao_interesse = [nome_coluna for nome_coluna in dados_treinamento.columns if nome_coluna not in colunas_interesse]

dados_treinamento.drop(columns=colunas_nao_interesse, inplace=True)
dados_teste.drop(columns=colunas_nao_interesse, inplace=True)

print('Shape dos datasets novos:')
print(dados_treinamento.shape)
print(dados_teste.shape)

### Conversão dos Dados para o Formato do Numpy

In [ ]:
dados_treinamento_np = dados_treinamento.to_numpy()

Y_treinamento = dados_treinamento_np[:,0]
X_treinamento = dados_treinamento_np[:,1:]

In [ ]:
dados_teste_np = dados_treinamento.to_numpy()

Y_teste = dados_teste_np[:,0]
X_teste = dados_teste_np[:,1:]

## Re-Seleção e Re-Aplicação de Modelos

### K-Nearest Neighbors

Para o KNN, vamos continuar considerando como parâmetros o número de vizinhos de 1 à 49 e as distâncias euclidiana, Manhattan, Minkowski e Chebyshev.

In [ ]:
parametros_grid = {
    'n_neighbors': [k for k in range(50) if k % 2 != 0],
    'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']   
}

modelo_knn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=parametros_grid,
    cv=10,
    scoring='accuracy',
    return_train_score=False,
    verbose=True,
    error_score='raise',
    n_jobs=-1,
)

In [ ]:
ajustar_modelo(modelo_knn)

In [ ]:
testar_modelo(modelo_knn)

### Naive Bayes

Para o Naive Bayes, vamos prosseguir novamente com o estimador gaussiano, com suavização testada de $1\cdot10^{-9}$ à $1$.

In [ ]:
parametros_grid = {
    'var_smoothing': np.logspace(0, -9, num=100),
}

modelo_bayes = GridSearchCV(
    estimator=GaussianNB(),
    param_grid=parametros_grid,
    cv=10,
    scoring='accuracy',
    return_train_score=False,
    verbose=True,
    error_score='raise',
    n_jobs=-1
)

In [ ]:
ajustar_modelo(modelo_bayes)

In [ ]:
testar_modelo(modelo_bayes)

### Multilayer Perceptron

Para o Multilayer Perceptron, foram testadas, novamente, 1 camada com 100 neurônios, ativações relu e logística com $\alpha = 0.01$.

In [ ]:
parametros_grid = {
    'hidden_layer_sizes': [(100,)],
    'activation': ['relu', 'logistic'],
    'alpha': [0.01],
}

modelo_perceptron = GridSearchCV(
    estimator=MLPClassifier(solver='sgd', random_state=SEMENTE_ALEATORIA, max_iter=1000),
    param_grid=parametros_grid,
    cv=10,
    scoring='accuracy',
    return_train_score=False,
    verbose=True,
    error_score='raise',
    n_jobs=-1
)

In [ ]:
ajustar_modelo(modelo_perceptron)

In [ ]:
testar_modelo(modelo_perceptron)

### Random Forest

Para o Random Forest, vamos continuar considerando estimadores de 100 à 300 (espaçados de 100 em 100), profundidade máxima de 5 à 25 (espaçadas de 5 em 5) e critérios gini e entropia para decisão.

In [ ]:
parametros_grid = {
    'n_estimators': [i * 100 for i in range(1, 4)],
    'max_depth': [i * 5 for i in range(1, 6)],
    'criterion': ['gini', 'entropy'],
}

modelo_forest = GridSearchCV(
    estimator=RandomForestClassifier(random_state=SEMENTE_ALEATORIA),
    param_grid=parametros_grid,
    cv=10,
    scoring='accuracy',
    return_train_score=False,
    verbose=True,
    error_score='raise',
    n_jobs=-1,
)

In [ ]:
ajustar_modelo(modelo_forest)

In [ ]:
testar_modelo(modelo_forest)

### Avaliação dos Desempenhos dos Novos Modelos

A nova aplicação dos algoritmos levou a uma piora nas métricas de acerto no KNN, a uma melhora no Naive Bayes e leve melhora no Random Forest. As métricas de acerto do Multilayer Perceptron permaneceram bastante semelhantes. Vale destacar que o percentual de acertos em cenários de empates, que já era relativamente bom no Random Forest, melhorou um pouco mais, mas permaneceu baixo nos outros modelos.

As métricas de avaliação obtidas nas predições para cada novo modelo podem ser sumarizadas na seguinte tabela:

|         Modelo        |                         Parâmetros                        | Acurácia (%) | Precisão (%) | Recall (%) | F1 (%) |
|:---------------------:|:---------------------------------------------------------:|:------------:|:------------:|:----------:|:------:|
|          KNN          |              Distância Euclideana, 43 Vizinhos             |      63      |      60      |     63     |   58   |
|      Naive Bayes      |                   Distribuição Gaussiana                  |      62      |      60      |     62     |   61   |
| Multilayer Perceptron |    Ativação Logística ($\alpha = 0.01)$, 100 Camadas    |      66      |      63      |     66     |   62   |
|     Random Forest     | Critério Gini, Profundidade Máxima de 10, 300 Estimadores |      81      |      81      |     81     |   80   |


## Aplicação nos Últimos Jogos

Vamos utilizar o modelo do classificador Random Forest para fazer predições nas 10 partidas da rodada ocorrida no último fim de semana (de 2 à 3 de dezembro de 2023). As partidas ocorridas foram:

|               Jogo               | Placar Final |
|:--------------------------------:|:------------:|
| Arsenal vs Wolves | 2 x 1 |
| Brentford vs Luton | 3 x 1 |
| Burnley vs Sheffield United | 5 x 0 |
| Nott'm Forest vs Everton | 0 x 1 |
| Newcastle vs Manchester United | 1 x 0 |
| Burnemouth vs Aston Villa | 2 x 2 |
| Chelsea vs Brighton | 3 x 2 |
| Liverpool vs Fulham | 4 x 3 |
| West Ham vs Crystal Palace | 1 x 1 |
| Manchester City vs Tottenham | 3 x 3 |

Os dados relativos à cada partida foram manualmente colocados no arquivo "ultimos-jogos.csv", já em um formato pronto para serem processados pelo modelo.

In [ ]:
dados_ultimos_jogos = pd.read_csv('./data/ultimos-jogos.csv')
dados_ultimos_jogos_np = dados_ultimos_jogos.to_numpy()

X_ultimos_jogos = dados_ultimos_jogos_np[:,1:]
Y_ultimos_jogos = np.array(dados_ultimos_jogos_np[:,0], dtype=int)

Y_previsto = np.array(modelo_forest.predict(X_ultimos_jogos), dtype=int)
dados_ultimos_jogos.insert(1, 'PR', Y_previsto)

acuracia = accuracy_score(Y_ultimos_jogos, Y_previsto)

display(dados_ultimos_jogos)
print('Acurácia:', acuracia)

O modelo previu corretamente o resultado de 6 partidas:

- Arsenal vs Wolves
- Brentford vs Luton
- Burnley vs Sheffield United
- Burnemouth vs Aston Villa
- Chelsea vs Brighton
- Liverpool vs Fulham

E errou outras 4 partidas:

- Nott'm Forest vs Everton (previu empate)
- Newcastle vs Manchester United (previu empate)
- West Ham vs Crystal Palace (previu visitante)
- Manchester City vs Tottenham (previu visitante)

## Conclusão

Ao longo deste projeto, foram explorados dados estatísticos de partidas da Premier League, utilizando diferentes modelos de aprendizado de máquina para tentar prever os resultados finais dos jogos. Os modelos avaliados foram KNN, Naive Bayes, Multilayer Perceptron e Random Forest. Apesar de a aplicação da Análise de Componentes Principais (PCA) ter sugerido certa complexidade na classificação, indicando a presença de variáveis intercorrelacionadas, o modelo Random Forest destacou-se, alcançando bons resultados em comparação com os demais. 

O resultado ao meio da partida se mostrou como o atributo que exerceu mais influência nas classificações, seguido de chutes (certos e totais) ao gol, escanteios, faltas cometidas e pertencimento dos grupos ao Big Six.

Uma observação interessante foi a dificuldade dos modelos em prever empates, sendo o Random Forest o mais eficaz nesse cenário. Exemplificando essa dificuldade, a seção "Aplicação nos Últimos Jogos" mostrou que, dentre as partidas erradas, 2 tiveram empates previstos incorretamente e as outras 2 tiveram um vencedor previsto incorretamente, enquanto o resultado ocorrido na prática foi de empate.

Sobre possíveis melhorias, é evidente a limitação na abordagem adotada para substituição de valores para as probabilidades faltantes de casas de apostas. Uma estratégia mais robusta poderia envolver uma substituição mais fundamentada, considerando resultados históricos e ajustando as probabilidades de vitória para os times do Big 6 também em partidas fora de casa.